<a href="https://colab.research.google.com/github/IsaacRider/HCO-Model-Project-Colab/blob/main/HCO-Single-Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Neuroscience Modeling Project
## Half Center Oscillator - Single Cell

### Setup before building the model
Only need to run the three cells below once.
#### Install NEURON
First install NEURON package.

In [1]:
!pip install neuron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 53.6 MB/s eta 0:00:00


#### Download modfiles from github

In [2]:
import os

if os.path.basename(os.getcwd())!='HCO-Model-Project-Colab':
  !git clone https://github.com/chenziao/HCO-Model-Project-Colab.git
  %cd HCO-Model-Project-Colab/
else:
  print('Mod files have already been downloaded.')
%ls

Cloning into 'HCO-Model-Project-Colab'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 43 (delta 20), reused 5 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 15.47 KiB | 3.09 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/HCO-Model-Project-Colab
capool.mod  cat.mod               HCO-Single-Cell.ipynb  hyper.mod   ka.mod   kdr.mod   na.mod
cas.mod     HCO_cell_template.py  HCO-Two-Cell.ipynb     inhsyn.mod  kca.mod  leak.mod


#### Compile modfiles

In [3]:
import os
print(os.system('nrnivmodl')) # compile modfiles. Return 0 for success, 1 for failure.

0


### Build the HCO model
Now you are ready to build the HCO model step by step, starting with a single cell.

#### Initialize the model
Run the cell below only once to import the cell template from the script "HCO_cell_template.py" and create a cell using the template. Then provide the cell with a current injection and a synapse as input to the cell.

In [4]:
from neuron import h
from HCO_cell_template import HCOcell
import matplotlib.pyplot as plt
import ipywidgets as widgets

h.load_file('stdrun.hoc')

h.dt = 0.025 # time step (resolution) of the simulation in ms
h.v_init= -50 # initial membrane potential in mV

# create a single cell from HCO_cell_template
cell = HCOcell()

# create a current clamp to the cell
ccl = h.IClamp(cell.soma(.5))

# create a synaptic input to the cell
syn = h.inhsyn(.5,sec=cell.soma) # create a synapse into soma
spk_train = h.NetStim() # provides a spike train
spk_train.noise = 0 # range 0 to 1, fractional randomness
nc = h.NetCon(spk_train,syn,0,0,1,sec=cell.soma) # a NEURON object that connects a source to a target

#### Set parameters and simulate
In the code below, you can change values for the parameters listed. The description of the parameters is provided in the comments (text after the symbol #).

Run the code after setting up the paremeter values, the simulation will run. After the simulation is done, run the code in the next step and you will see the result plots and the values of the parameters being used printed out.

You can change the parameters and run the code below repeatedly to tune the model.

In [44]:
h.tstop = 10000 # Simulation time (ms)

# Biophysical parameters
parameters = {
    # Conductance of channels (siemens/cm2)
    'gbar_leak': None,
    'gbar_na': .2,    # (.1~.5)
    'gbar_kdr': .1,    # (.1~.5)
    'gbar_ka': 0.15,    # (.1~.5)
    'gbar_kca': 0.039,    # (.01~.05)
    'gbar_cas': 0.01,    # (.001~.01)
    'gbar_cat': 0.009,    # (.005~.01)
    'gbar_hyper': 0.,    # (.0001~.0003)
    # Ca pool parameters
    'tauca_capool': None,    # decay time constant
    'fca_capool': None    # ca influx factor that goes to ca pool
} # parameters set to "None" are to use default values defined in the template

# Current injection parameters
ccl_param = {
    'delay': 200,    # start time (ms)
    'dur': 800,    # duration (ms)
    'amp': 1.0    # amplitude (nA) (set to 0 to disable current injection)
}

# Synaptic input parameters
syn_param = {
    'weight': 0,    # synaptic weight (set to 0 to disable, 1 to enable synapse)
    'start': 100.,    # start time of first spike (ms)
    'interval': 500.,    # time between spikes (ms)
    'number': 5,    # number of spikes
    'esyn': -40,    # synaptic channels reversal potential (mV) (set to -80)
    'gmax': 40e-3,    # synaptic channels maximum conductance (uS) (default: 40e-3)
    'tau1': 10,    # rise time (ms) (default: 10)
    'tau2': 20    # decay time (ms) (default: 20)
}


# Setup parameters (Do Not need to change the code below)
cell.set_biophysics(**parameters)
for p,v in ccl_param.items():
    setattr(ccl,p,v)
syn_objs = [nc,spk_train,syn]
obj_id = [0,1,1,1,2,2,2,2]
for i,(p,v) in zip(obj_id,syn_param.items()):
    if i>0:
        setattr(syn_objs[i],p,v)
    else:
        getattr(syn_objs[i],p)[0] = v

# Run simulation
_ = h.run()

#### Plot results
After simulation is done in previous step, run the code below to plot the results.

In [45]:
out1 = widgets.Output()
out2 = widgets.Output()
# Plot results
with out1:
    fig,axs = cell.plot_vars(figsize=(16,8))
    # axs[1].set_ylim(-0.2,0.2) # uncomment this line and select range to zoom in y-axis (e.g.-0.2~0.2)
    plt.show()
# Print parameters
with out2:
    print('Parameters:')
    for p,v in cell.get_biophysics(**parameters).items():
        print('{} = {}'.format(p,v))
widgets.HBox([out1,out2])